In [ ]:
# Korean Sentence Splitter 설치
!pip install kss

In [ ]:
# KPF-BERT-NER의 ner class 모음
!wget https://raw.githubusercontent.com/KPF-bigkinds/BIGKINDS-LAB/main/KPF-BERT-NER/label.py

--2024-07-24 05:22:49--  https://raw.githubusercontent.com/KPF-bigkinds/BIGKINDS-LAB/main/KPF-BERT-NER/label.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11245 (11K) [text/plain]
Saving to: ‘label.py’

label.py            100%[===================>]  10.98K  --.-KB/s    in 0s      

2024-07-24 05:22:49 (103 MB/s) - ‘label.py’ saved [11245/11245]



기사 데이터인 "2024.csv" 로드   
valid_ner.py 로드

In [ ]:
import pandas as pd

df = pd.read_csv("2024.csv")
print(len(df))
df.head()

32294


,id,title,date,content,link,norm_title,norm_content
0,122846231,中 군 고위간부 9명 전인대 대표 박탈… ‘로켓군’ 납품 관련 부패사건 연루 의혹,20240101,“리상푸 前국방부장 낙마와 연관” 리부장 후임엔 해군출신 둥쥔 중국 당국이 ‘반(反...,https://www.donga.com/news/Inter/article/all/2...,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,"""리상푸 전국방부장 낙마와 연관"" 리부장 후임엔 해군출신 둥쥔 중국 당국이 '반(반..."
1,122846362,서울 12cm 기습 폭설… “이런 눈 또 온다”,20240101,13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 “엘니뇨-온난화로 폭...,https://www.donga.com/news/Society/article/all...,"서울 12cm 기습 폭설… ""이런 눈 또 온다""","13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 ""엘니뇨-온난화로 폭..."
2,122846400,"‘자산 301조원’ 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,https://www.donga.com/news/People/article/all/...,"'자산 301조원' 머스크, 세계 최고 부자에",1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...
3,122846403,나이지리아서 피랍 17일만에… 대우건설 韓직원 2명 풀려나,20240101,나이지리아에서 무장단체에 납치됐던 대우건설 한국인 직원 2명이 17일 만에 무사히 ...,https://www.donga.com/news/Economy/article/all...,나이지리아서 피랍 17일만에… 대우건설 한직원 2명 풀려나,나이지리아에서 무장단체에 납치됐던 대우건설 한국인 직원 2명이 17일 만에 무사히 ...
4,122846406,"부모 학대로 다리 잃은 英 9세 소년, 최연소 훈장 받는다",20240101,혼자 의족-목발로 10km 걷기 도전 자기 이름 딴 재단 세워 32억원 모금 어린이...,https://www.donga.com/news/People/article/all/...,"부모 학대로 다리 잃은 영 9세 소년, 최연소 훈장 받는다",혼자 의족-목발로 10km 걷기 도전 자기 이름 딴 재단 세워 32억원 모금 어린이...


In [ ]:
from transformers import AutoTokenizer, BertForTokenClassification, logging
logging.set_verbosity_error()
import sys, os, torch
import numpy as np
import label # label.py를 업로드 한 후
import kss

print(torch.cuda.is_available()) # 리소스를 GPU 계열로 선택한 이후

True


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("KPF/KPF-bert-ner")
model = AutoModelForTokenClassification.from_pretrained("KPF/KPF-bert-ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/455M [00:00<?, ?B/s]

In [ ]:
def ner_predict(text):
    text = text.replace('\n','')
    model.to("cuda")

    sents = kss.split_sentences(text)
    decoding_ner_sentence = ""
    word_list = list()
    pred_str = list()

    #text to model input
    for idx, sent in enumerate(sents):

        sent = sent.replace(" ", "-")
        test_tokenized = tokenizer(sent, return_tensors="pt")

        test_input_ids = test_tokenized["input_ids"].to("cuda")
        test_attention_mask = test_tokenized["attention_mask"].to("cuda")
        test_token_type_ids = test_tokenized["token_type_ids"].to("cuda")

        inputs = {
            "input_ids" : test_input_ids,
            "attention_mask" : test_attention_mask,
            "token_type_ids" : test_token_type_ids
        }

        if inputs['input_ids'].size()[1] > 512:
            cnt = int(inputs['input_ids'].size()[1])

            inp_np = inputs['input_ids'].cpu().numpy()
            att_np = inputs['attention_mask'].cpu().numpy()
            tok_np = inputs['token_type_ids'].cpu().numpy()

            for i in range(cnt):
                slice_inp = inp_np[0][(i*512):((i+1)*512)]
                slice_att = att_np[0][(i * 512):((i + 1) * 512)]
                slice_tok = tok_np[0][(i * 512):((i + 1) * 512)]

                slice_inp = slice_inp.reshape(1, len(slice_inp))
                slice_att = slice_att.reshape(1, len(slice_att))
                slice_tok = slice_tok.reshape(1, len(slice_tok))

                slice_inp = torch.tensor(slice_inp)
                slice_att = torch.tensor(slice_att)
                slice_tok = torch.tensor(slice_tok)

                slice_inp = torch.tensor(slice_inp).to("cuda")
                slice_att = torch.tensor(slice_att).to("cuda")
                slice_tok = torch.tensor(slice_tok).to("cuda")

                slice_inputs = {
                    "input_ids": slice_inp,
                    "attention_mask": slice_att,
                    "token_type_ids": slice_tok
                }

                # predict
                outputs = model(**slice_inputs)

                token_predictions = outputs[0].argmax(dim=2)
                token_prediction_list = token_predictions.squeeze(0).tolist()

                pred = [label.id2label[l] for l in token_prediction_list]

                pred_str = np.concatenate((pred_str, pred))
        else:
            #predict
            outputs = model(**inputs)

            token_predictions = outputs[0].argmax(dim=2)
            token_prediction_list = token_predictions.squeeze(0).tolist()

            pred_str = [label.id2label[l] for l in token_prediction_list]
        tt_tokenized = tokenizer(sent).encodings[0].tokens

        # decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False
        _word = ""
        # word_list = list()

        #model output to text
        for i, (token, pred) in enumerate(zip(tt_tokenized, pred_str)):
            if i == 0 or i == len(pred_str) - 1:
                continue
            token = token.replace('#', '').replace("-", " ")

            if token == "":
                continue

            if 'B-' in pred:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'
                    word_list.append({"word" : _word, "label" : prev_entity_tag, "desc" : "1"})
                    _word = ""

                if token[0] == ' ':
                    token = list(token)
                    token[0] = ' <'
                    token = ''.join(token)
                    decoding_ner_sentence += token
                    _word += token
                else:
                    decoding_ner_sentence += '<' + token
                    _word += token
                is_prev_entity = True
                prev_entity_tag = pred[2:]
                is_there_B_before_I = True

            elif 'I-' in pred:
                decoding_ner_sentence += token
                _word += token

                if is_there_B_before_I is True:
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token
                    is_prev_entity = False
                    is_there_B_before_I = False
                    word_list.append({"word" : _word, "label" : prev_entity_tag, "desc" : label.ner_code[prev_entity_tag]})
                    _word = ""
                else:
                    decoding_ner_sentence += token

    # print("OUTPUT")
    # print("sentence : ", decoding_ner_sentence)
    # print("result : ", word_list)
    return word_list

In [ ]:
from valid_ner import valid_ner_code
import re
from tqdm import tqdm

# ner_origin = [] # NER 원본
ner_custom = [] # 전처리된 NER
contents = df['norm_content'].tolist()
for content in tqdm(contents):
    valid_ner = []
    try:
      ner_results = ner_predict(content)
      # ner_origin.append(ner_results)

      for ner_result in ner_results:
          if ner_result['label'] in list(valid_ner_code.keys()): # valid한 NER 클래스이면
              if (ner_result['label'] == 'PS_NAME') and (len(ner_result['word']) == 1):
                  continue
              if ner_result['word'] == "대표":
                  continue
              if '월' in ner_result['word'] or '일' in ner_result['word']: # '12월 31일중국' 이런 식의 오류가 있다
                  pattern1 = r'\d{1,2}월'
                  pattern2 = r'\d{1,2}일'

                  # 정규 표현식을 사용하여 날짜 부분 제거
                  ner_result['word'] = re.sub(pattern1, '', ner_result['word'])
                  ner_result['word'] = re.sub(pattern2, '', ner_result['word']).replace("  ", " ").strip()
              # 유효한 NER만 뽑아서 리스트에 넣음
              valid_ner.append(ner_result['word'])
    except:
      valid_ner.append('-')
    ner_custom.append(valid_ner)

  0%|          | 12/32294 [00:02<1:56:03,  4.64it/s]<ipython-input-11-665f187d73d1>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  slice_inp = torch.tensor(slice_inp).to("cuda")
<ipython-input-11-665f187d73d1>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  slice_att = torch.tensor(slice_att).to("cuda")
<ipython-input-11-665f187d73d1>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  slice_tok = torch.tensor(slice_tok).to("cuda")
100%|██████████| 32294/32294 [2:31:42<00:00,  3.55it/s]


In [ ]:
len(ner_custom)

32294

In [ ]:
for x in ner_custom[:10]:
  print(x)

['리상푸', '전국방부장', '리부장', '해군', '둥쥔', '중국', '군', '중국', '국방부장', '대만', '필리핀', '남중국해', '중국', '신화통신', '중국', '국회', '로켓군', '부사령원', '중앙군', '연합참모부', '부참모장인', '장전중', '장진중', '미사일', '로켓군', '커우젠원', '구건문', '대만정치대', '주임', '전국인대', '중국', '중앙군사위', '중국', '리상푸', '李상복', '국방부장', '장관', '둥쥔', '동군', '인민해방군', '해군', '사령원', '해군참모총장', '국방부장', '산둥성', '옌타이', '해군', '부장', '인민해방군', '전략', '미사일', '로켓군', '주임', '부장', '인민해방군', '중국', '우옌성', '오연생', '중국항공우주과학기술그룹', '회장', '중화권', '우첸', '오겸', '국방부', '대변인', '베이징', '김기용', '특파원']
['서울', '기상청', '서울', '한반도', '수도권', '강원', '서울', '광진구', '마을버스', '종로구', '독립문역', '서대문구', '서대문역', '통일로', '5차로', '천호대교', '북단', '강원', '춘천시', '트럭', '강원', '삼척시', '아래', '구급대원', '평창군', '소방대원', '기상청', '북쪽', '남쪽', '기상청', '김백민', '부경대', '교수', '한반도', '최미송', '기자', 'cms@donga.com', '주애진', '기자']
['아르노', 'LVMH', '회장', '베이조스', '게이츠', '머어이재용', '회장', '일론', '프랑스', '루이뷔통모에에네시', 'LVMH', '베르나르', '아르노', '회장', '블룸버그', '일론', '머스크', '테슬라', '최고', 'CEO', '머스크', 'LVMH', '아르노', '회장', 'LVMH', '제프', '베이조스', '아마존', '게이츠', '마이크로소프트', 'MS', '스티브', 

In [ ]:
cnt = 0
for x in ner_custom:
  if (x == []) or (x[0] == '-'):
      cnt += 1
print(cnt)

6297


In [ ]:
for idx in range(len(ner_custom)):
  if (ner_custom[idx] == []):
    ner_custom[idx].append('-')

In [ ]:
df['NER'] = ner_custom
df.columns

Index(['id', 'title', 'date', 'content', 'link', 'norm_title', 'norm_content',
       'NER'],
      dtype='object')

In [ ]:
df.head()

,id,title,date,content,link,norm_title,norm_content,NER
0,122846231,中 군 고위간부 9명 전인대 대표 박탈… ‘로켓군’ 납품 관련 부패사건 연루 의혹,20240101,“리상푸 前국방부장 낙마와 연관” 리부장 후임엔 해군출신 둥쥔 중국 당국이 ‘반(反...,https://www.donga.com/news/Inter/article/all/2...,중 군 고위간부 9명 전인대 대표 박탈… '로켓군' 납품 관련 부패사건 연루 의혹,"""리상푸 전국방부장 낙마와 연관"" 리부장 후임엔 해군출신 둥쥔 중국 당국이 '반(반...","[리상푸, 전국방부장, 리부장, 해군, 둥쥔, 중국, 군, 중국, 국방부장, 대만,..."
1,122846362,서울 12cm 기습 폭설… “이런 눈 또 온다”,20240101,13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 “엘니뇨-온난화로 폭...,https://www.donga.com/news/Society/article/all...,"서울 12cm 기습 폭설… ""이런 눈 또 온다""","13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 ""엘니뇨-온난화로 폭...","[서울, 기상청, 서울, 한반도, 수도권, 강원, 서울, 광진구, 마을버스, 종로구..."
2,122846400,"‘자산 301조원’ 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,https://www.donga.com/news/People/article/all/...,"'자산 301조원' 머스크, 세계 최고 부자에",1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,"[아르노, LVMH, 회장, 베이조스, 게이츠, 머어이재용, 회장, 일론, 프랑스,..."
3,122846403,나이지리아서 피랍 17일만에… 대우건설 韓직원 2명 풀려나,20240101,나이지리아에서 무장단체에 납치됐던 대우건설 한국인 직원 2명이 17일 만에 무사히 ...,https://www.donga.com/news/Economy/article/all...,나이지리아서 피랍 17일만에… 대우건설 한직원 2명 풀려나,나이지리아에서 무장단체에 납치됐던 대우건설 한국인 직원 2명이 17일 만에 무사히 ...,"[나이지리아, 대우건설, 한국인, 대우건설, 나이지리아, 바이엘사주, 한국인, 운전..."
4,122846406,"부모 학대로 다리 잃은 英 9세 소년, 최연소 훈장 받는다",20240101,혼자 의족-목발로 10km 걷기 도전 자기 이름 딴 재단 세워 32억원 모금 어린이...,https://www.donga.com/news/People/article/all/...,"부모 학대로 다리 잃은 영 9세 소년, 최연소 훈장 받는다",혼자 의족-목발로 10km 걷기 도전 자기 이름 딴 재단 세워 32억원 모금 어린이...,"[친부모, 6주양쪽, 다리, 토니, 허젤, 영국, 영국, BBC, 찰스, 3세, 국..."


In [ ]:
df.to_csv("2024_ner.csv")